# Download the model

Download the full trained TF Model

In [1]:
!wget -nc https://github.com/DataTalksClub/machine-learning-zoomcamp/releases/download/chapter7-model/xception_v4_large_08_0.894.h5 -O clothing-model.h5


File ‘clothing-model.h5’ already there; not retrieving.


# Convert to saved_model format

In [2]:
import tensorflow as tf
from tensorflow import keras

model = keras.models.load_model('./clothing-model.h5')

tf.saved_model.save(model, 'clothing-model')

INFO:tensorflow:Assets written to: clothing-model/assets


INFO:tensorflow:Assets written to: clothing-model/assets


In [3]:
!tree clothing-model

clothing-model
├── assets
├── fingerprint.pb
├── saved_model.pb
└── variables
    ├── variables.data-00000-of-00001
    └── variables.index

3 directories, 4 files


In [4]:
# !ls -lhR clothing-model

## Take a look on the saved model using tensorflow's built-in utility `saved_model_cli`

In [5]:
!saved_model_cli show --dir clothing-model --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['input_8'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 299, 299, 3)
      name: serving_default_input_8:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['dense_7'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 10)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


# Connect to TF serving

In [6]:
import grpc
import tensorflow as tf
import json
 
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

In [7]:
# Set the address and port of the TensorFlow Serving server
server_address = 'localhost'
server_port = 8500

# Create a gRPC channel
channel = grpc.insecure_channel(f'{server_address}:{server_port}')

# Create a stub for the prediction service
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [8]:
from io import BytesIO
from urllib import request

import numpy as np

from PIL import Image

def preprocess_input(x):
    x /= 127.5
    x -= 1
    return x


def preprocess_img(img):
    x = np.array(img.resize((299, 299)), dtype="float32")
    X = np.expand_dims(x, axis=0)
    X = preprocess_input(X)
    return X


def read_img_from_url(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)

    return preprocess_img(img)

In [9]:
url = 'http://bit.ly/mlbookcamp-pants'

X = read_img_from_url(url)

In [10]:
classes = [
    "dress",
    "hat",
    "longsleeve",
    "outwear",
    "pants",
    "shirt",
    "shoes",
    "shorts",
    "skirt",
    "t-shirt",
]

In [11]:
input_tensor_name = 'input_8'
output_tensor_name = 'dense_7'
model_name = 'clothing-model'

# Create a request object
request = predict_pb2.PredictRequest()

# Set the model name and input tensor data
request.model_spec.name = 'clothing-model'
request.model_spec.signature_name = tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY

# Serialize the example proto to bytes and set it in the request
request.inputs[input_tensor_name].CopyFrom(
    tf.make_tensor_proto(X, dtype=tf.float32)
)


In [12]:
# Make the gRPC request and get the response
response = stub.Predict(request, timeout=10.0)

# Extract and print the result 
output_data = response.outputs[output_tensor_name].float_val

print(f"Model Prediction: {json.dumps(dict(zip(classes, output_data)), indent=4)}")

Model Prediction: {
    "dress": -1.9792001247406006,
    "hat": -4.7238240242004395,
    "longsleeve": -2.130927324295044,
    "outwear": -1.0491342544555664,
    "pants": 9.957743644714355,
    "shirt": -3.027919054031372,
    "shoes": -3.8510289192199707,
    "shorts": 3.5252106189727783,
    "skirt": -2.746251106262207,
    "t-shirt": -4.712244510650635
}


# Test `gateway.py`

Run tf_serving docker-compose:

```shell
docker compose -f docker-compose-tf-serving.yaml up -d
```

Launch it with 
```shell
python gateway.py
```

Run a curl
```shell
curl -X POST 'http://127.0.0.1:8000/predict'  -H 'Content-Type: application/json' -d '{  
  "url": "http://bit.ly/mlbookcamp-pants"
}'
```

or go to the Swagger UI

http://127.0.0.1:8000/docs#/default/predict_predict_post